In [ ]:
############## PLEASE RUN THIS CELL FIRST! ###################

# import everything and define a test runner function
from importlib import reload
from helper import run
import network

from block import GENESIS_BLOCK_HASH
from helper import calculate_new_bits
from network import (
    NetworkEnvelope,
    VersionMessage,
)

### Exercise 1

Determine what this network message is:

`f9beb4d976657261636b000000000000000000005df6e0e2`

In [ ]:
# Exercise 1

message_hex = 'f9beb4d976657261636b000000000000000000005df6e0e2'

# convert to binary
# see what the bytes 4 through 16 aregi

### Exercise 2

Write the `parse` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_parse`

In [ ]:
# Exercise 2

reload(network)
run(network.NetworkEnvelopeTest("test_parse"))

### Exercise 3

Write the `serialize` method for `NetworkEnvelope`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:NetworkEnvelopeTest:test_serialize`

In [ ]:
# Exercise 3

reload(network)
run(network.NetworkEnvelopeTest("test_serialize"))

### Exercise 4

Write the `serialize` method for `VersionMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:VersionMessageTest:test_serialize`

In [ ]:
# Exercise 4

reload(network)
run(network.VersionMessageTest("test_serialize"))

### Exercise 5

Write the `handshake` method for `SimpleNode`

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:SimpleNodeTest:test_handshake`

In [ ]:
# Exercise 5

reload(network)
run(network.SimpleNodeTest("test_handshake"))

### Exercise 6

Write the `serialize` method for `GetHeadersMessage`.

#### Make [this test](/edit/code-ch10/network.py) pass: `network.py:GetHeadersMessageTest:test_serialize`

In [ ]:
# Exercise 6

reload(network)
run(network.GetHeadersMessageTest("test_serialize"))

In [ ]:
from network import SimpleNode, GetHeadersMessage, HeadersMessage
from block import GENESIS_BLOCK_HASH
from helper import calculate_new_bits
node = SimpleNode('btc.programmingblockchain.com', testnet=False)
node.handshake()
last_block_hash = GENESIS_BLOCK_HASH
first_epoch_block = None
expected_bits = None
count = 1
for _ in range(19):
    getheaders = GetHeadersMessage(start_block=last_block_hash)
    node.send(getheaders)
    headers = node.wait_for(HeadersMessage)
    for b in headers.blocks:
        if not b.check_pow():
            raise RuntimeError('bad PoW at block {}'.format(count))
        if last_block_hash != GENESIS_BLOCK_HASH and b.prev_block != last_block_hash:
            raise RuntimeError('discontinuous block at {}'.format(count))
        if expected_bits and b.bits != expected_bits:
            raise RuntimeError('bad bits at block {}'.format(count))
        if first_epoch_block and count % 2016 == 2015:
            expected_bits = calculate_new_bits(
                expected_bits, b.timestamp - first_epoch_block.timestamp)
            print(expected_bits.hex())
        elif first_epoch_block is None:
            expected_bits = b.bits
        if count % 2016 == 0 or not first_epoch_block:
            first_epoch_block = b
        count += 1
        last_block_hash = b.hash()